In [4]:
import datetime
import time
import psycopg2

url = "hdfs://127.0.0.1:9000/socmed_input/"
out_url = "hdfs://127.0.0.1:9000/socmed_output/"

conn = psycopg2.connect(database="bigdatatest",
    host="localhost",
    user="geraldakbar",
    password='bigdata'
)

In [1]:
def normalizeTime(time):
    res = ''
    for i in range(len(time)):
        if i == 6 or i == 7:
            res += '0'
        elif i == 4:
            if int(time[i]) < 5:
                res += '0'
            else:
                res += '5'
        else:
            res += time[i] 
    return res

def parseTime(type, time):
    if type == "twitter":
        ts = t.strftime('%Y-%m-%d %H:%M:%S', t.strptime(time,'%a %b %d %H:%M:%S +0000 %Y'))
        return ts[:10]
    elif type == "instagram" or type == "other":
        time = str(datetime.datetime.fromtimestamp(int(time)).date())
        return time[:10]
    else:
        return time[:10] + ' ' + normalizeTime(time[11:19])

In [3]:
fb = spark.read.json(f"{url}facebook*.json").rdd

def map(line):
    result = []
    postDate = line["created_time"][0:10] + ' ' + normalizeTime(line["created_time"][11:19])
    comments = line["comments"]["data"]
    result.append((("facebook", postDate), 1))
    for comment in comments:
        date = comment["created_time"][0:10] + ' ' + normalizeTime(comment["created_time"][11:19])
        result.append((("facebook", date), 1))
    return result

def userMap(line):
    user = line["from"]["name"]
    postDate = line["created_time"][0:10] + ' ' + normalizeTime(line["created_time"][11:19])
    return (("facebook", postDate, user), 1)
    

cursor = conn.cursor()
fbPost = fb.flatMap(map)
fbPost = fbPost.reduceByKey(lambda x, y: x + y)

user = fb.map(userMap).distinct().map(lambda x: ((x[0][0], x[0][1]), x[1]))
user = user.reduceByKey(lambda x, y: x + y)

fbPost = fbPost.join(user).map(lambda x: (x[0], x[1][0], x[1][1]))
fbPost = fbPost.map(lambda x: (x[0][0], x[0][1], x[1], x[2]))
fbPost.take(5)

[('facebook', '2022-01-01 15:15:00', 33, 1),
 ('facebook', '2022-01-01 01:35:00', 2, 1),
 ('facebook', '2021-12-29 08:55:00', 1, 1),
 ('facebook', '2021-12-28 10:40:00', 2, 1),
 ('facebook', '2021-12-24 06:15:00', 1, 1)]

23/04/17 11:17:01 WARN spark.HeartbeatReceiver: Removing executor driver with no recent heartbeats: 966665 ms exceeds timeout 120000 ms
23/04/17 11:17:01 ERROR scheduler.TaskSchedulerImpl: Lost executor driver on localhost: Executor heartbeat timed out after 966665 ms
23/04/17 11:17:01 WARN spark.SparkContext: Killing executors is not supported by current scheduler.
23/04/17 11:17:01 WARN storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_6_piece0 !
23/04/17 11:17:01 WARN storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_4_piece0 !
23/04/17 11:17:01 WARN storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_2_piece0 !
23/04/17 11:17:01 WARN storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_5_piece0 !
23/04/17 11:17:01 WARN storage.BlockManagerMasterEndpoint: No more replicas available for broadcast_3_piece0 !
